## BaggingClassifier + TF-IDF 

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
stop_words = stopwords.words('english')

In [2]:
training_data = pd.read_csv('train.csv')

#### Combining the 3 columns ( keyword + location + text ) - filling the NAN by a blank or ''

In [3]:
training_data['text'] = training_data['keyword'].fillna('') + training_data['location'].fillna('')  \
                        + training_data['text'].fillna('')
training_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
training_data = training_data.drop(columns=['id','keyword','location'],axis=1)
training_data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


#### Data Cleaning step involves tokenizing, removing the stopwords and removing non-alphanumeric tokens

In [5]:
def clean_data(text):
    tokens = text.split()
    no_stopwords = [x for x in tokens if x not in stop_words]
    no_nonalphanum = [x.lower() for x in no_stopwords if x.isalnum()]
    return ' '.join(no_nonalphanum)

In [6]:
training_data['text'] = training_data['text'].apply(clean_data)
print(training_data.shape)
training_data.head()

(7613, 2)


,text,target
0,our deeds reason may allah forgive us,1
1,forest fire near la ronge canada,1
2,all residents asked notified no evacuation she...,1
3,people receive evacuation orders california,1
4,just got sent photo ruby smoke pours school,1


In [7]:
test_data = pd.read_csv('test.csv')
test_id = test_data['id']
test_data['text'] = test_data['keyword'].fillna('') + test_data['location'].fillna('')  \
                        + test_data['text'].fillna('')
test_data = test_data.drop(columns=['id','keyword','location'],axis=1)
test_data['text'] = test_data['text'].apply(clean_data)
print(test_data.shape)
test_data.head()

(3263, 1)


,text
0,just happened terrible car crash
1,heard different stay safe
2,forest fire spot geese fleeing across i cannot...
3,apocalypse
4,typhoon soudelor kills 28 china taiwan


In [8]:
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf

#### download pre-trained word embeddings model

In [9]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [10]:
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

#### creating neural network of 4 layers, having a word embeddings layer

In [11]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
x_train,x_test,y_train,y_test = train_test_split(training_data.text,training_data.target)

In [14]:
history = model.fit(x_train,y_train,epochs=50)

Train on 5709 samples
Epoch 1/50
5709/5709 [==============================] - 2s 317us/sample - loss: 0.6007 - accuracy: 0.6716
Epoch 2/50
5709/5709 [==============================] - 1s 229us/sample - loss: 0.5126 - accuracy: 0.7518
Epoch 3/50
5709/5709 [==============================] - 1s 228us/sample - loss: 0.4773 - accuracy: 0.7730
Epoch 4/50
5709/5709 [==============================] - 1s 226us/sample - loss: 0.4461 - accuracy: 0.7893
Epoch 5/50
5709/5709 [==============================] - 1s 234us/sample - loss: 0.4159 - accuracy: 0.8075
Epoch 6/50
5709/5709 [==============================] - 1s 243us/sample - loss: 0.3869 - accuracy: 0.8224
Epoch 7/50
5709/5709 [==============================] - 1s 228us/sample - loss: 0.3588 - accuracy: 0.8336
Epoch 8/50
5709/5709 [==============================] - 1s 226us/sample - loss: 0.3315 - accuracy: 0.8520
Epoch 9/50
5709/5709 [==============================] - 1s 225us/sample - loss: 0.3050 - accuracy: 0.8648
Epoch 10/50
5709/5709 [=

In [15]:
results = model.evaluate(x_test,y_test, verbose=2)

1904/1904 - 0s - loss: 1.8421 - accuracy: 0.7206


In [16]:
predictions = model.predict_classes(x_test)
accuracy_score(y_test,predictions)

0.7205882352941176

In [17]:
history = model.fit(training_data.text,training_data.target,epochs=50)

Train on 7613 samples
Epoch 1/50
7613/7613 [==============================] - 2s 270us/sample - loss: 0.4061 - accuracy: 0.9087
Epoch 2/50
7613/7613 [==============================] - 2s 273us/sample - loss: 0.2491 - accuracy: 0.9211
Epoch 3/50
7613/7613 [==============================] - 2s 281us/sample - loss: 0.2069 - accuracy: 0.9280
Epoch 4/50
7613/7613 [==============================] - 2s 278us/sample - loss: 0.1840 - accuracy: 0.9320
Epoch 5/50
7613/7613 [==============================] - 2s 277us/sample - loss: 0.1673 - accuracy: 0.9379
Epoch 6/50
7613/7613 [==============================] - 2s 292us/sample - loss: 0.1550 - accuracy: 0.9415
Epoch 7/50
7613/7613 [==============================] - 2s 293us/sample - loss: 0.1439 - accuracy: 0.9450
Epoch 8/50
7613/7613 [==============================] - 2s 301us/sample - loss: 0.1349 - accuracy: 0.9471
Epoch 9/50
7613/7613 [==============================] - 2s 307us/sample - loss: 0.1261 - accuracy: 0.9510
Epoch 10/50
7613/7613 [=

In [18]:
test_data = test_data['text']
predictions = model.predict_classes(test_data)

In [19]:
predictions = predictions.reshape(-1)
results = pd.DataFrame({
                        'id':test_id,
                        'target':predictions
                        })
results.to_csv('submissions.csv',index=False)

#### Accuracy :---
#####                - Local Accuracy : 73.52 (over split training data)
#####                - Online Accuracy : 72.59 (After fitting over all training data)